In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
df= pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Drop irrelevant columns
df.drop(['RowNumber','CustomerId','Surname'], axis=1, inplace=True)

In [4]:
# Encode categorical variables
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])

In [5]:
# Encoding Geography column using one-hot encoding
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
geo_ohe_encoder = ohe.fit_transform(df[['Geography']])
geo_ohe_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
geo_ohe_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [7]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
encoded_df = pd.DataFrame(geo_ohe_encoder.toarray(),columns=ohe.get_feature_names_out(['Geography'])).head()

In [9]:
df.drop('Geography', axis=1, inplace=True)


In [10]:
df = pd.concat([df, encoded_df],axis=1)
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,NaN,NaN,NaN
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,NaN,NaN,NaN
9997,709,0,36,7,0.00,1,0,1,42085.58,1,NaN,NaN,NaN
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,NaN,NaN,NaN


In [11]:
pickle.dump(label_encoder,open('label_encoder.pkl','wb'))
pickle.dump(ohe,open('geo_ohe_encoder.pkl','wb'))

In [12]:
# Splitting the dataset into training and testing sets

X=df.drop('Exited',axis=1)
y=df['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [13]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,         nan,
                nan,         nan],
       [-0.20389777,  0.91324755,  0.29493847, ...,         nan,
                nan,         nan],
       [-0.96147213,  0.91324755, -1.41636539, ...,         nan,
                nan,         nan],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,         nan,
                nan,         nan],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,         nan,
                nan,         nan],
       [ 0.47065475,  0.91324755,  1.15059039, ...,         nan,
                nan,         nan]])

In [14]:
X_test

array([[-0.57749609,  0.91324755, -0.6557859 , ...,         nan,
                nan,         nan],
       [-0.29729735,  0.91324755,  0.3900109 , ...,         nan,
                nan,         nan],
       [-0.52560743, -1.09499335,  0.48508334, ...,         nan,
                nan,         nan],
       ...,
       [ 0.81311987, -1.09499335,  0.77030065, ...,         nan,
                nan,         nan],
       [ 0.41876609,  0.91324755, -0.94100321, ...,         nan,
                nan,         nan],
       [-0.24540869,  0.91324755,  0.00972116, ...,         nan,
                nan,         nan]])

In [15]:
pickle.dump(scaler,open('scaler.pkl','wb'))

In [16]:
# ANN Implementattion
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [17]:
# Build our ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # 1st hidden layer
    Dense(32, activation='relu'), # 2nd hidden layer
    Dense(1, activation='sigmoid') # Output layer
]
)

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

In [20]:
#compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics='accuracy')

In [21]:
#Setup the tensorboard 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [22]:
#Setup early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [23]:
#train the model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping, tensorflow_callback])

Epoch 1/100


250/250 [==============================] - 6s 11ms/step - loss: 0.5469 - accuracy: 0.7945 - val_loss: 0.4969 - val_accuracy: 0.8035
Epoch 2/100
250/250 [==============================] - 2s 7ms/step - loss: 0.5081 - accuracy: 0.7945 - val_loss: 0.4961 - val_accuracy: 0.8035
Epoch 3/100
250/250 [==============================] - 2s 6ms/step - loss: 0.5081 - accuracy: 0.7945 - val_loss: 0.4957 - val_accuracy: 0.8035
Epoch 4/100
250/250 [==============================] - 1s 6ms/step - loss: 0.5081 - accuracy: 0.7945 - val_loss: 0.4959 - val_accuracy: 0.8035
Epoch 5/100
250/250 [==============================] - 1s 6ms/step - loss: 0.5081 - accuracy: 0.7945 - val_loss: 0.4955 - val_accuracy: 0.8035
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 0.5082 - accuracy: 0.7945 - val_loss: 0.4956 - val_accuracy: 0.8035
Epoch 7/100
250/250 [==============================] - 1s 6ms/step - loss: 0.5082 - accuracy: 0.7945 - val_loss: 0.4958 - val_accuracy: 0.8

In [24]:
model.save('ann_model.h5')

c:\Users\abhishek\practise_ml\neural network\ann classification\.venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
#Load tensorboard Extension
%load_ext tensorboard

In [26]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 5160), started 3:41:45 ago. (Use '!kill 5160' to kill it.)